### Imports

In [16]:
import pandas as pd
import numpy as np
import requests
import logging
import ipywidgets as widgets
from utils.mongo_conn import MongoConnect
from IPython.core.display import display
from tqdm import tqdm

In [5]:
logging.basicConfig(filename='./logs/drugs_errors.log', level=logging.INFO)

### Connection to MongoDB

In [32]:
recall_mongo_conn = MongoConnect('team_G_db_mongo_final_1', 'drugs_recalled')
recall_collection = recall_mongo_conn.connect()

drugs_event_conn = MongoConnect('team_G_db_mongo_final_1', 'drugs_event')
event_collection = drugs_event_conn.connect()

label_mongo_conn = MongoConnect('team_G_db_mongo_final_1', 'Drugs_Product_Label')
drug_product_labelling = label_mongo_conn.connect()

### Requests through API

In [4]:
status_label = widgets.Label('Requesting Data through API. This could take some time. Please wait!')
display(status_label)

Label(value='Requesting Data through API. This could take some time. Please wait!')

In [33]:
####Requesting Data through API from FDA for all the three data sources
recalled_data_url = 'https://api.fda.gov/drug/enforcement.json?api_key=jzBDRKRffIUITo0uKg9EEWqasttW2cDni2b7ncUB&limit=1000&skip='
event_url = 'https://api.fda.gov/drug/event.json?api_key=ERMRbxTchMkRnHapeAM1Q0qVlXnNUXTIbFUE7948&limit=1000&skip='
label_url = 'https://api.fda.gov/drug/label.json?api_key=GSwiympJkhfjzpbGwZp9GBrtS0caLj9E4UBSsYPq&limit=1000&skip='
f = widgets.IntProgress(min=0, max=25, bar_style='info') # instantiate the bar
display(widgets.HBox([widgets.Label('Progress:'),f]))
try:
    success=True
    for skip_records in range(0,25000, 1000):
        if skip_records<13000:
            url_1 = recalled_data_url + str(skip_records)
            response_recall = requests.get(url_1)
            if response_recall.status_code == 200:
                json_to_dump = response_recall.json()['results']
                recall_collection.insert_many(json_to_dump)
            else:
                success=False
                logging.info("Failed at mongo_connection", exc_info=True)
                break
            url_2 = label_url+ str(skip_records)
            response_label = requests.get(url_2)
            if response_label.status_code == 200:
                json_to_dump_label = response_label.json()['results']
                drug_product_labelling.insert_many(json_to_dump_label)
            else:
                success=False
                logging.info("Failed at mongo_connection", exc_info=True)
                break
        url = event_url + str(skip_records)
        response = requests.get(url)
        if response.status_code == 200:
            json_to_dump = response.json()['results']
            event_collection.insert_many(json_to_dump)
        else:
            success=False
            logging.info("Failed at mongo_connection", exc_info=True)
            break
        f.value+=1
    if success:
        f.bar_style='success'
        status_label.value='Load to MongoDB Successfull. Proceeding with Data Cleaning.'
    else:
        status_label.value='Failed to dump data to Mongo'
except ConnectionError:
    print('Unable to connect to source!. Try again after some time.')
except:
    print('Oh No, something went wrong! Check logs for info!')
    logging.error("Exception occurred at mongo_connection", exc_info=True)